# DINO: DETR with Improved DeNoising Anchor Boxes for End-to-End Object Detection

https://github.com/IDEA-Research/DINO

[Papers With Code Link](https://paperswithcode.com/paper/focal-modulation-networks)

In [1]:
!pip install --quiet pylance duckdb torch torchvision transforms

## Build and install [DINO]() Model

In [2]:
!git -C DINO pull || git clone https://github.com/IDEACVR/DINO
%cd DINO

!pip install --quiet -r requirements.txt \
  && cd models/dino/ops \
  && python setup.py -q build install

Already up to date.
/home/lei/work/lance/python/notebooks/DINO
/home/lei/work/lance/python/venv/lib/python3.10/site-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/home/lei/work/lance/python/venv/lib/python3.10/site-packages/torch/utils/cpp_extension.py:387: UserWarning: The detected CUDA version (11.8) has a minor version mismatch with the version that was used to compile PyTorch (11.7). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/home/lei/work/lance/python/venv/lib/python3.10/site-packages/torch/utils/cpp_extension.py:397: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 11.8
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_ver

In [3]:
# See https://github.com/IDEA-Research/DINO/blob/main/inference_and_visualization.ipynb
# for instruction to load model
from util.slconfig import SLConfig
from main import build_model_main
model_config_path = "config/DINO/DINO_4scale.py"

args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 
model, criterion, postprocessors = build_model_main(args)

/home/lei/work/lance/python/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lei/work/lance/python/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Downloads weights

# Download DINO-4scale weights
! [[ -f /tmp/model.pt ]] || gsutil cp gs://eto-public/models/dino/checkpoint0033_4scale.pth /tmp/model.pt
import torch
model_checkpoint_path = "/tmp/model.pt"
checkpoint = torch.load(model_checkpoint_path)
model.load_state_dict(checkpoint['model'])
_ = model.cuda().eval()

## Prepare COCO validation dataset to [Lance](https://github.com/eto-ai/lance) format.

In [7]:
! [[ -f annotations/instances_val2017.json ]] || ( \
  wget -O /tmp/annotations.zip http://images.cocodataset.org/annotations/annotations_trainval2017.zip && \
  unzip -o -qq /tmp/annotations.zip && rm annotations.zip \
)
! [[ -d val2017/ ]] || ( \
  wget -O /tmp/val2017.zip http://images.cocodataset.org/zips/val2017.zip && \
  unzip -o -qq /tmp/val2017.zip && \
  rm val2017.zip )

import pandas as pd
import json
from lance.data.convert.coco import load_detection

val_data = load_detection("val2017", "annotations/instances_val2017.json")
val_data


dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])


In [ ]:
from lance.pytorch import Dataset
import torchvision.transforms as T
import pandas as pd

transform = T.Compose([
    T.Resize(400),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

threshold = 0.5

dataset = Dataset(
    "s3://eto-public/datasets/coco/coco.lance",
    columns=["image", "split", "image_id"],
    # mode="batch",
    batch_size=8)
results = []
with torch.no_grad():
  for batch in dataset:
    image_id = batch[2].cpu().item()
    imgs = [transform(batch[0]).cuda()]
    # print(batch, batch.shape)
    output = model(imgs)
    output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]
    mask = output["scores"] > threshold
    pred = {
        "image_id": image_id,
        "boxes": output["boxes"][mask].cpu().numpy(),
        "labels": output["labels"][mask].cpu().numpy(),
        "scores": output["scores"][mask].cpu().numpy(),
    }
    del output
    results.append(pred)

df = pd.DataFrame(data=results)
df